In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# import packages
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import matplotlib
plt.style.use('ggplot')
from matplotlib.pyplot import figure

%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (12,8)

In [ ]:
# read the data
#flights 
df= pd.read_csv('../input/flight-delays/flights.csv')


In [ ]:
#VitesseMoyenneAvion
df['VitesseMoyenneAvion']=df['DISTANCE']*60/df['AIR_TIME']
df['WEATHER_DELAY'].fillna(int(0), inplace=True)
df['LATE_AIRCRAFT_DELAY'].fillna(int(0), inplace=True)
df['SECURITY_DELAY'].fillna(int(0), inplace=True)
df['AIRLINE_DELAY'].fillna(int(0), inplace=True)
df['AIR_SYSTEM_DELAY'].fillna(int(0), inplace=True)

#ConsomSuppCarb
df['ConsomSuppCarb']=(df['AIR_SYSTEM_DELAY']+df['WEATHER_DELAY']+df['LATE_AIRCRAFT_DELAY']+df['SECURITY_DELAY']+df['AIRLINE_DELAY'])*60*5.5
#Moyenne TaxiOut par aéroport / Jointure avec la table Flights
Moyenne_TaxiOut_par_aéroport=df[['ORIGIN_AIRPORT','TAXI_OUT']].groupby('ORIGIN_AIRPORT')['TAXI_OUT'].mean()
df=df.join(Moyenne_TaxiOut_par_aéroport, on='ORIGIN_AIRPORT', rsuffix='_Moyenne')
           #Moyenne TaxiIN par aéroport / Jointure avec la table Flights
Moyenne_TaxiIN_par_aéroport=df[['DESTINATION_AIRPORT','TAXI_IN']].groupby('DESTINATION_AIRPORT')['TAXI_IN'].mean()
df=df.join(Moyenne_TaxiIN_par_aéroport, on='DESTINATION_AIRPORT', rsuffix='_Moyenne')
df['TaxiInSupp']=df['TAXI_IN']-df['TAXI_IN_Moyenne']
df['TaxiOutSupp']=df['TAXI_OUT']-df['TAXI_OUT_Moyenne']
import gc
del df['TAXI_IN_Moyenne']
gc.collect()
del df['TAXI_OUT_Moyenne']
gc.collect()
df['SCHEDULED_ARRIVAL'].fillna(0, inplace=True)
df['ARRIVAL_TIME'].fillna(0, inplace=True)
df.loc[df['CANCELLED'] == 1, 'Statut'] = 'Annulé' 
df.loc[df['DIVERTED'] == 1, 'Statut'] = 'Dérouté'
df.loc[(df['ARRIVAL_TIME'].astype(int)-df['SCHEDULED_ARRIVAL'].astype(int)<=0) & (df['CANCELLED'] != 1) & (df['DIVERTED'] != 1) , 'Statut'] = 'A temps'
df.loc[(df['ARRIVAL_TIME'].astype(int)-df['SCHEDULED_ARRIVAL'].astype(int)>0) & (df['CANCELLED'] != 1) & (df['DIVERTED'] != 1) , 'Statut'] = 'Retardé'
df['CANCELLATION_REASON'].fillna("E", inplace=True)
df.loc[df['CANCELLATION_REASON'] == "A", 'FactAnnulation'] = 'Airline/Carrier' 
df.loc[df['CANCELLATION_REASON'] == "B", 'FactAnnulation'] = 'Weather'
df.loc[df['CANCELLATION_REASON'] == "C", 'FactAnnulation'] = 'National Air System'
df.loc[df['CANCELLATION_REASON'] == "D", 'FactAnnulation'] = 'Security'
df.loc[df['CANCELLATION_REASON'] == "E", 'FactAnnulation'] = 'Aucun'

In [ ]:
#Retard Total pour chaque vol
df['RetardTotal']=df['AIR_SYSTEM_DELAY']+df['WEATHER_DELAY']+df['LATE_AIRCRAFT_DELAY']+df['SECURITY_DELAY']+df['AIRLINE_DELAY']

In [ ]:
df['DATE']=df['YEAR'].astype(str)+"-"+df['MONTH'].astype(str)+"-"+df['DAY'].astype(str)

In [ ]:
df1=df[df["AIRLINE"]=="UA"]

In [ ]:
import gc
del df
gc.collect()

In [ ]:
df1.head()

ESSAYONS DE PREDIRE LES RETARDS QUE LES VOLS de la compagnies aérienne "Spirit Air Lines"VONT SUBIR.

Choix du modèle:

> **Processus Autorégressif**

Le Processus Autorégressif est un modèle de régression pour séries temporelles.

Contrairement à une regression linéaire, la série temporelle est expliquée par ses valeurs passées plutôt que par d'autres variables.

Dans notre cas, il permettrait de prédire le retard des vols en fonction des retards enregistrées auparavant.

Avantage: Les retards observés durant les différents vols changent en fonction de la période de l'année, comme on l'a démontré au début de l'analyse. De même, cette compagnie aérienne est celle qui enregistre le plus de taux de retards. Cette prédiction servirait non seulement aux clients pour programmer leur vols un peu à l'avance, mais aussi à la compagnie aérienne elle-même pour bien gérer la répartition de ses ressources en fonction des périodes où le service subit de la pression et donc cause des retards.


In [ ]:
#Tout d'abord on calcule corrélation entre le retard des vols de cette compagnie à t et retard à à t-1
from pandas.plotting import lag_plot
lag_plot(df1['RetardTotal'],lag=1)

In [ ]:
#coefficient de correlation  de Pearson
series = df1['RetardTotal']
values = pd.DataFrame(series.values)
dataframe = pd.concat([values.shift(1), values], axis=1)
dataframe.columns = ['t-1', 't']
print(dataframe)
print()
result = dataframe.corr()
print(result)

On remarque que les points sont très dispersés sur le graphe. De plus, le coefficient de corrélation étant égal à 0.08, très proche de 0 donc absence totale de corrélation, cette variable à t et à t-1 sont linéairement indépendantes.

L'application du processus autorégressif sur ce modèle donnerait de faux résultats.

In [ ]:
#Visualisons la corrélation entre les différents variables.
sns.pairplot(data = df1[["RetardTotal","TaxiOutSupp","TaxiInSupp","ConsomSuppCarb","VitesseMoyenneAvion"]])

In [ ]:
sns.pairplot(data = df1[['LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY']])

In [ ]:
sns.pairplot(data = df1[['VitesseMoyenneAvion','ConsomSuppCarb']])

In [ ]:
df1.columns

On remarque d'après le diagramme qu'il y a probablement une relation entre le retard et la consommation supplémentaire de carburant par l'avion.

On va essayer maintenant de prédire la consommation supplémentaire de carburant en fonction du retard effectué par l'avion en question.

Pour cela on va procéder en utilisant une régression linéaire et l'optimisation des paramètres se fera à travers la descente de gradient.

La fonction de prédiction: h(x) = theta(0) + theta(1) * x

On cherche à trouver le meilleur couple (theta(0),theta(1)) de telle manière à minimiser le coût global des erreurs unitaires.

In [ ]:
values1 = pd.DataFrame(df1["ConsomSuppCarb"].values)
values2= pd.DataFrame(df1["RetardTotal"].values)
dataframe = pd.concat([values1, values2], axis=1)
dataframe.columns = ['ConsomSuppCarb', 'RetardTotal']
print(dataframe[dataframe['ConsomSuppCarb']!=0])

In [ ]:
np.savetxt("Predict.txt", dataframe.values, fmt='%d',delimiter=',')